In [2]:
import numpy as np
import pandas as pd
import numba as nb

#### Monte carlo simulations

In [5]:
def call_payoff(S:float, K:float, r:float, q:float, T:float) -> float:
    return np.maximum(S*np.exp(-q*T)-K*np.exp(-r*T), 0)

def put_payoff(S:float, K:float, r:float, q:float, T:float) -> float:
    return np.maximum(K*np.exp(-r*T)-S*np.exp(-q*T), 0)

def monte_carlo_vectorized(S0:float, K:float, sigma:float, rf:float, q:float, t:float, NOofsteps:int, NOofPaths:int, seed:int = 42) -> pd.DataFrame:
    
    r = rf - q
    np.random.seed(seed)
    random = np.random.normal(0, 1, size = (NOofsteps, NOofPaths))
    dt=t/NOofsteps
    exp_ = np.exp((r - (sigma**2)/2)*dt + sigma*random*np.sqrt(dt))
    exp_cumprod = np.cumprod(exp_, axis=0)
    paths = exp_cumprod*S0
    paths[0,:] = S0
    
    S_t = paths[-1, :]

    S_t_call = call_payoff(S_t, K, rf, q, t)
        
    S_t_put = put_payoff(S_t, K,  rf, q, t)
    
    
    return paths, S_t_call, S_t_put

# longstaff and schwartz (2001) approach to value American Options

def longstaff_schwartz(opt_type:str, path : np.array, exercise_payoff_func, K:float, r:float, q:float, T:float, bf:int = 2):
    # Discount Factors considering exercise throughout the option life
    df = np.array([(T - i/252) for i in reversed(range(path.shape[0]))])
    # get a (t x n) matrix, t:days, n: number of paths
    df = np.tile(df, (path.shape[1],1)).T
    
    # intrinsic values not accounting the discount factors
    intr_value = exercise_payoff_func(path, K, r, q, 0)
    # discounted at time 0 payoff
    V_disc = exercise_payoff_func(path, K, r, q, df)
    # terminal payoffs 
    V = V_disc[-1, :]

    for i in reversed(range(1, path.shape[0] - 1)):
        x = path[i, : ]
        # fit curve
        if opt_type == "put":
            otm = np.where(x > K)
            itm = np.where(x < K)
        
        elif opt_type == "call":
            otm = np.where(x < K)
            itm = np.where(x > K)
            
        itm_x = np.delete(x, otm)
        itm_o = np.delete(V, otm)
        
        try:
            rg = np.polyfit(itm_x,  itm_o, bf)
            C = np.polyval(rg, itm_x)
            
        except Exception:
            continue
         
        V[itm] = np.where(V_disc[i, itm] > C, V_disc[i, itm], V[itm])
    
    return V
         

####  Pathwise Greeks

In [19]:
def Pathwise_delta(opt_type, S0, K, path, r, t):
    temp_1 = path[-1, :] > K
    call_delta = np.exp(-r*t)*np.mean(path[-1:,]/S0*temp_1)
    if opt_type.lower() == "call":
        return call_delta
    elif opt_type.lower() == "put":
        return -(1-call_delta)

def Pathwise_vega(S0, K, path, sigma, r, t):
    temp_1 = path[-1, :] > K
    temp_2 = 1.0/sigma * path[-1, :]*(np.log(path[-1, :]/S0) - (r + 0.5*sigma**2.0)*t)
    return np.exp(-r*T)*np.mean(temp_1*temp_2)


#DELTA
def finite_diff_delta(opt_type:str, dS:float, S0:float, K:float, sigma:float, rf:float, q:float, t:float, NOofsteps:int, NOofPaths:int, seed:int = 42) -> float:
    # _, call, put = monte_carlo_vectorized(S0, K, sigma, rf, q, t, NOofsteps, NOofPaths, seed)
    _, call_plus, put_plus = monte_carlo_vectorized(S0 + dS, K, sigma, rf , q, t , NOofsteps, NOofPaths, seed)
    _, call_minus, put_minus = monte_carlo_vectorized(S0 - dS, K, sigma, rf , q, t , NOofsteps, NOofPaths, seed)
    
    if opt_type.lower() == "call":
        delta = (np.mean(call_plus) - np.mean(call_minus))/2*(dS)
    elif opt_type.lower() == "put":
        delta = (np.mean(put_plus) - np.mean(put_minus))/2*(dS)
    
    return delta

finite_diff_delta = nb.jit(finite_diff_delta)

#GAMMA
def finite_diff_gamma(opt_type:str, dS:float, S0:float, K:float, sigma:float, rf:float, q:float, t:float, NOofsteps:int, NOofPaths:int, seed:int = 42) -> float:
    _, call, put = monte_carlo_vectorized(S0, K, sigma, rf, q, t, NOofsteps, NOofPaths, seed)
    _, call_plus, put_plus = monte_carlo_vectorized(S0 + dS, K, sigma, rf , q, t , NOofsteps, NOofPaths, seed)
    _, call_minus, put_minus = monte_carlo_vectorized(S0 - dS, K, sigma, rf , q, t , NOofsteps, NOofPaths, seed)
    
    if opt_type.lower() == "call":
        gamma = (np.mean(call_plus) - 2*np.mean(call) + np.mean(call_minus))/(dS)**2
    elif opt_type.lower() == "put":
        gamma = (np.mean(put_plus) - 2*np.mean(put) + np.mean(put_minus))/(dS)**2
    
    return gamma

finite_diff_gamma = nb.jit(finite_diff_gamma)

#RHO
def finite_diff_rho(opt_type:str, dr:float, S0:float, K:float, sigma:float, rf:float, q:float, t:float, NOofsteps:int, NOofPaths:int, seed:int = 42) -> float:
    # _, call, put = monte_carlo_vectorized(S0, K, sigma, rf, q, t, NOofsteps, NOofPaths, seed)
    _, call_plus, put_plus = monte_carlo_vectorized(S0, K, sigma, rf + dr, q, t , NOofsteps, NOofPaths, seed)
    _, call_minus, put_minus = monte_carlo_vectorized(S0, K, sigma, rf - dr, q, t , NOofsteps, NOofPaths, seed)
    
    if opt_type.lower() == "call":
        rho = (np.mean(call_plus) - np.mean(call_minus))/2*dr
    elif opt_type.lower() == "put":
        rho = (np.mean(put_plus) - np.mean(put_minus))/2*dr
    
    return rho

finite_diff_rho = nb.jit(finite_diff_rho)

#THETA
def finite_diff_theta(opt_type:str, dt:float, S0:float, K:float, sigma:float, rf:float, q:float, t:float, NOofsteps:int, NOofPaths:int, seed:int = 42) -> float:
    # _, call, put = monte_carlo_vectorized(S0, K, sigma, rf, q, t, NOofsteps, NOofPaths, seed)
    _, call_plus, put_plus = monte_carlo_vectorized(S0, K, sigma, rf, q, t + dt, NOofsteps, NOofPaths, seed)
    _, call_minus, put_minus = monte_carlo_vectorized(S0, K, sigma, rf, q, t - dt, NOofsteps, NOofPaths, seed)
    
    if opt_type.lower() == "call":
        theta =(np.mean(call_plus) - np.mean(call_minus))/2*dt
    elif opt_type.lower() == "put":
        theta = (np.mean(put_plus) - np.mean(put_minus))/2*dt
    
    return theta

finite_diff_theta = nb.jit(finite_diff_theta)